In [0]:
from pyspark import SparkConf, SparkContext
import json
import pandas as pd
import csv
import os

In [0]:
!pwd

/content


In [0]:
conf = (
    SparkConf()
    .setAppName("your app name")
    .set("spark.driver.memory", "40g")
    .set("spark.executor.memory", "40g")
)
sc = SparkContext(conf=conf)

Read in terminal 2 data files 189-211

In [0]:
#=======INPUT: PATH TO FILE WITH LAT/LONG AND RATINGS OF EACH LOCATION=======#
locations_info = 'Location Data (1).csv'

#=======INPUT: PATH TO DATA FOLDER=======#
path="/Users/jerrytong/Desktop/USC/inf 560/tps2_181-211"
file_path=os.listdir(path)
file_path.remove('.DS_Store')
file_path.remove('.ipynb_checkpoints')

Convert all files to a RDD 

In [0]:
f1=sc.union([sc.textFile(path+ "/" + f) for f in file_path])

In [0]:
t2=f1.map(lambda x: json.loads(x))

In [0]:
t2.take(2)

[{'Building': 'TPS2',
  'Level': 'Level 3',
  'ClientMacAddr': 'ca:86:6c:ca:15:1b',
  'lat': -22.813743585939573,
  'lng': -43.247113277534446,
  'localtime': '2019-12-20 10:23:27.003 UTC'},
 {'Building': 'TPS2',
  'Level': 'Level 3',
  'ClientMacAddr': 'ca:86:6c:ca:15:1b',
  'lat': -22.813743585939573,
  'lng': -43.247113277534446,
  'localtime': '2019-12-20 10:23:27.003 UTC'}]

Filter out distinct December data

In [0]:
Dec_data=t2.filter(lambda x: '2019-12' in x['localtime']).map(lambda x: (x['Building'],x['Level'],x['ClientMacAddr'],round(x['lat'],5),round(x['lng'],5),x['localtime'])).distinct()

In [0]:
Dec_data.count()
#26370585 rows in total

26370585

Group by macid alone

A particular person's route in the airport in december -- this leads to calculation of distance between two locations(say gate and restaurant) and the time spent between this two location

In [0]:
Macid=Dec_data.groupBy(lambda x:x[2]).flatMap(lambda x:list(x[1]))  #groupby Macid

Read google map location data in, convert location to float 

In [0]:
location=sc.textFile(locations_info)

In [0]:
Google_loc=location.map(lambda x: x.split(',')).map(lambda x:((float(x[0]),float(x[1])),float(x[2])))

In [0]:
Google_loc.take(3)

[((-22.8145, -43.2457), 4.1),
 ((-22.8147, -43.2466), 3.2),
 ((-22.8148, -43.2467), 4.7)]

Groupby latitude and longitude of terminal2 December data and round up to 4 digits

In [0]:
Lat_long=Dec_data.groupBy(lambda x: (x[3],x[4])).flatMap(lambda x : list(x[1]))

In [0]:
Lat_long_round_up=Lat_long.map(lambda x: ((round(x[3],4),round(x[4],4)),x))

In [0]:
Lat_long_round_up.take(3)

[((-22.8149, -43.2465),
  ('TPS2',
   'Level 3',
   '9c:ad:97:5c:24:d1',
   -22.81489,
   -43.2465,
   '2019-12-01 05:06:01.249 UTC')),
 ((-22.8149, -43.2465),
  ('TPS2',
   'Level 3',
   'da:0f:99:23:d7:ea',
   -22.81489,
   -43.2465,
   '2019-12-06 02:48:51.287 UTC')),
 ((-22.8149, -43.2465),
  ('TPS2',
   'Level 3',
   '50:92:b9:d6:a4:26',
   -22.81489,
   -43.2465,
   '2019-12-08 15:02:22.037 UTC'))]

Merge Kiana's terminal 2 data with Google map location data to extract locations

In [0]:
merge=Google_loc.join(Lat_long_round_up)

In [0]:
merge.take(3)

[((-22.8145, -43.247),
  (4.2,
   ('TPS2',
    'Level 3',
    'c0:11:73:c6:2f:0a',
    -22.81448,
    -43.24699,
    '2019-12-08 11:11:48.016 UTC'))),
 ((-22.8145, -43.247),
  (4.2,
   ('TPS2',
    'Level 3',
    '38:53:9c:a8:52:e7',
    -22.81448,
    -43.24699,
    '2019-12-11 05:45:22.291 UTC'))),
 ((-22.8145, -43.247),
  (4.2,
   ('TPS2',
    'Level 3',
    'bc:98:df:b1:6a:98',
    -22.81448,
    -43.24699,
    '2019-12-21 08:29:20.752 UTC')))]

Extract only rating,level,macid and timestamp from merged data by location longitude and latitude

In [0]:
merge1=merge.map(lambda x:(x[0],(x[1][0],x[1][1][1],x[1][1][2],x[1][1][5])))

In [0]:
merge1.take(3)

[((-22.8145, -43.247),
  (4.2, 'Level 3', 'c0:11:73:c6:2f:0a', '2019-12-08 11:11:48.016 UTC')),
 ((-22.8145, -43.247),
  (4.2, 'Level 3', '38:53:9c:a8:52:e7', '2019-12-11 05:45:22.291 UTC')),
 ((-22.8145, -43.247),
  (4.2, 'Level 3', 'bc:98:df:b1:6a:98', '2019-12-21 08:29:20.752 UTC'))]

Groupby longitude and latitude to find out location informations

In [0]:
merg=merge1.groupByKey().map(lambda x : (x[0], list(x[1])))

Display 26 locations we have

In [0]:
lat_long=merg.map(lambda x: x[0]).take(26)

In [0]:
lat_long

[(-22.8145, -43.247),
 (-22.8153, -43.2448),
 (-22.8139, -43.2476),
 (-22.815, -43.2445),
 (-22.8137, -43.2472),
 (-22.8151, -43.2472),
 (-22.815, -43.2471),
 (-22.8155, -43.2454),
 (-22.8142, -43.2471),
 (-22.8148, -43.2473),
 (-22.8145, -43.2457),
 (-22.8154, -43.2462),
 (-22.8151, -43.2465),
 (-22.8134, -43.2469),
 (-22.8141, -43.2471),
 (-22.8148, -43.2467),
 (-22.814, -43.2477),
 (-22.8152, -43.2446),
 (-22.8139, -43.2471),
 (-22.8136, -43.2475),
 (-22.815, -43.2453),
 (-22.8148, -43.247),
 (-22.8147, -43.2466),
 (-22.815, -43.2457),
 (-22.8152, -43.2467),
 (-22.8153, -43.2461)]

location info for all 26 places(contains rating, level,macid,timestamp of all visits to 26 locations)

In [0]:
merg3=merg.map(lambda x: x[1]).collect()

Convert 26 location information to dataframe and write out to csv file

In [0]:
for i in range(len(merg3)):
  dataIndex = i + 1
  csvName = 'data' + str(dataIndex) + '.csv'
  csvFile = pd.DataFrame(merg3[i]).to_csv(csvName)

example of location 1 info

In [0]:
data1=merg3[0]
data1

[(4.2, 'Level 3', 'c0:11:73:c6:2f:0a', '2019-12-08 11:11:48.016 UTC'),
 (4.2, 'Level 3', '38:53:9c:a8:52:e7', '2019-12-11 05:45:22.291 UTC'),
 (4.2, 'Level 3', 'bc:98:df:b1:6a:98', '2019-12-21 08:29:20.752 UTC'),
 (4.2, 'Level 3', 'd0:04:01:4a:6b:8c', '2019-12-24 21:42:56.517 UTC'),
 (4.2, 'Level 3', '70:fd:46:a6:1d:40', '2019-12-08 22:54:11.41 UTC'),
 (4.2, 'Level 3', 'c0:a5:dd:17:fb:e6', '2019-12-22 03:31:51.356 UTC'),
 (4.2, 'Level 3', 'c0:a5:dd:17:fb:e6', '2019-12-22 21:05:05.903 UTC'),
 (4.2, 'Level 3', 'c0:a5:dd:17:fb:e6', '2019-12-28 14:45:27.529 UTC'),
 (4.2, 'Level 3', '90:73:5a:5e:9a:b4', '2019-12-10 02:46:51.148 UTC'),
 (4.2, 'Level 3', '5c:af:06:9d:63:62', '2019-12-17 10:47:03.766 UTC'),
 (4.2, 'Level 3', 'fc:64:3a:24:ab:dc', '2019-12-19 01:03:21.749 UTC'),
 (4.2, 'Level 3', 'b8:27:eb:6f:c2:6f', '2019-12-30 05:44:48.101 UTC'),
 (4.2, 'Level 3', '88:03:55:7a:59:6c', '2019-12-02 18:59:34.598 UTC'),
 (4.2, 'Level 3', '0c:80:63:25:be:91', '2019-12-04 08:29:04.003 UTC'),
 (4.2, 